In [7]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "mytext.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
print(chars)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])


Total Characters:  7918
Total Vocab:  52
['\n', ' ', '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '=', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '“', '”']
[24, 25, 42, 43, 41, 24, 26, 43, 21, 29, 44, 35, 35, 1, 24, 37, 30, 35, 28, 1, 42, 39, 24, 43, 32, 24, 35, 1, 26, 38, 36, 39, 38, 44, 37, 27, 32, 37, 30, 1, 32, 42, 1, 38, 29, 1, 32, 37, 43, 28, 41, 28, 42, 43, 1, 29, 38, 41, 0, 32, 36, 24, 30, 32, 37, 30, 1, 42, 36, 24, 35, 35, 1, 38, 41, 30, 24, 37, 42, 1, 2, 25, 41, 28, 24, 42, 43, 5, 1, 43, 28, 42, 43, 32, 26, 35, 28, 42, 3, 1]
[25, 42, 43, 41, 24, 26, 43, 21, 29, 44, 35, 35, 1, 24, 37, 30, 35, 28, 1, 42, 39, 24, 43, 32, 24, 35, 1, 26, 38, 36, 39, 38, 44, 37, 27, 32, 37, 30, 1, 32, 42, 1, 38, 29, 1, 32, 37, 43, 28, 41, 28, 42, 43, 1, 29, 38, 41, 0, 32, 36, 24, 30, 32, 37, 30, 1, 42, 36, 24, 35, 35, 1, 38, 41, 30, 24, 37, 42, 1, 2, 25,

[28, 1, 35, 38, 26, 24, 43, 32, 38, 37, 1, 38, 29, 1, 43, 31, 28, 1, 39, 38, 32, 37, 43, 1, 42, 26, 24, 43, 43, 28, 41, 28, 41, 7, 1, 43, 31, 28, 0, 28, 29, 29, 28, 26, 43, 1, 38, 29, 1, 43, 31, 28, 1, 42, 39, 28, 28, 27, 1, 38, 29, 1, 42, 38, 44, 37, 27, 1, 45, 24, 41, 32, 24, 43, 32, 38, 37, 1, 24, 37, 27, 1, 43, 31, 28, 1, 42, 26, 24, 43, 43, 28, 41, 28, 41, 1, 35, 38, 26, 24]
[1, 35, 38, 26, 24, 43, 32, 38, 37, 1, 38, 29, 1, 43, 31, 28, 1, 39, 38, 32, 37, 43, 1, 42, 26, 24, 43, 43, 28, 41, 28, 41, 7, 1, 43, 31, 28, 0, 28, 29, 29, 28, 26, 43, 1, 38, 29, 1, 43, 31, 28, 1, 42, 39, 28, 28, 27, 1, 38, 29, 1, 42, 38, 44, 37, 27, 1, 45, 24, 41, 32, 24, 43, 32, 38, 37, 1, 24, 37, 27, 1, 43, 31, 28, 1, 42, 26, 24, 43, 43, 28, 41, 28, 41, 1, 35, 38, 26, 24, 43]
[35, 38, 26, 24, 43, 32, 38, 37, 1, 38, 29, 1, 43, 31, 28, 1, 39, 38, 32, 37, 43, 1, 42, 26, 24, 43, 43, 28, 41, 28, 41, 7, 1, 43, 31, 28, 0, 28, 29, 29, 28, 26, 43, 1, 38, 29, 1, 43, 31, 28, 1, 42, 39, 28, 28, 27, 1, 38, 29, 1, 42, 3

[36, 42, 1, 22, 11, 23, 1, 24, 37, 27, 1, 24, 35, 32, 30, 37, 32, 37, 30, 0, 43, 31, 28, 1, 32, 37, 27, 32, 45, 32, 27, 44, 24, 35, 1, 24, 6, 35, 32, 37, 28, 42, 1, 24, 35, 38, 37, 30, 1, 43, 31, 28, 42, 28, 1, 39, 24, 43, 31, 42, 1, 22, 12, 23, 7, 1, 43, 31, 32, 42, 1, 24, 39, 39, 41, 38, 24, 26, 31, 5, 0, 31, 38, 46, 28, 45, 28, 41, 5, 1, 41, 28, 40, 44, 32, 41, 28, 42, 1, 24]
[42, 1, 22, 11, 23, 1, 24, 37, 27, 1, 24, 35, 32, 30, 37, 32, 37, 30, 0, 43, 31, 28, 1, 32, 37, 27, 32, 45, 32, 27, 44, 24, 35, 1, 24, 6, 35, 32, 37, 28, 42, 1, 24, 35, 38, 37, 30, 1, 43, 31, 28, 42, 28, 1, 39, 24, 43, 31, 42, 1, 22, 12, 23, 7, 1, 43, 31, 32, 42, 1, 24, 39, 39, 41, 38, 24, 26, 31, 5, 0, 31, 38, 46, 28, 45, 28, 41, 5, 1, 41, 28, 40, 44, 32, 41, 28, 42, 1, 24, 37]
[1, 22, 11, 23, 1, 24, 37, 27, 1, 24, 35, 32, 30, 37, 32, 37, 30, 0, 43, 31, 28, 1, 32, 37, 27, 32, 45, 32, 27, 44, 24, 35, 1, 24, 6, 35, 32, 37, 28, 42, 1, 24, 35, 38, 37, 30, 1, 43, 31, 28, 42, 28, 1, 39, 24, 43, 31, 42, 1, 22, 12, 23

In [13]:
# Load LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "mytext.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
filename = "weights-improvement-19-2.8432.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Total Characters:  7918
Total Vocab:  52
Total Patterns:  7818
Seed:
" ly coherent nature of the
echosonography, it may be reduced if the data is compounded
incoherently.  "
oh the the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th 

In [1]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "mytext.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
print(X)
print(n_vocab)


Using TensorFlow backend.


Total Characters:  7918
Total Vocab:  52
Total Patterns:  7818
[[[24]
  [25]
  [42]
  ..., 
  [42]
  [ 3]
  [ 1]]

 [[25]
  [42]
  [43]
  ..., 
  [ 3]
  [ 1]
  [24]]

 [[42]
  [43]
  [41]
  ..., 
  [ 1]
  [24]
  [37]]

 ..., 
 [[41]
  [24]
  [42]
  ..., 
  [ 1]
  [10]
  [18]]

 [[24]
  [42]
  [38]
  ..., 
  [10]
  [18]
  [18]]

 [[42]
  [38]
  [44]
  ..., 
  [18]
  [18]
  [12]]]
52


In [2]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "mytext.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print(len(y))
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
filename = "weights-improvement-30-1.7248-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Total Characters:  7918
Total Vocab:  52
Total Patterns:  7818
7818
Seed:
" ical
object of diameter 6 cm containing four polypropylene
fibres stretched orthogonal to the imagin "
 of the soeed of sound distribution as she soaetie cmmation and the cxial correction as auielt acooe of the soeed of sound distribution as she soaetie cmmation and the cxial correction as auielt acooe of the soeed of sound distribution as she soaetie cmmation and the cxial correction as auielt acooe of the soeed of sound distribution as she soaetie cmmation and the cxial correction as auielt acooe of the soeed of sound distribution as she soaetie cmmation and the cxial correction as auielt acooe of the soeed of sound distribution as she soaetie cmmation and the cxial correction as auielt acooe of the soeed of sound distribution as she soaetie cmmation and the cxial correction as auielt acooe of the soeed of sound distribution as she soaetie cmmation and the cxial correction as auielt acooe of the soeed of soun